In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Evaluation - Brain MRI Metastasis Segmentation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from torch.utils.data import DataLoader\n",
    "from torchvision import transforms\n",
    "from tqdm import tqdm\n",
    "\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "from src.data.dataset import BrainMRIDataset\n",
    "from src.models.nested_unet import NestedUNet\n",
    "from src.models.attention_unet import AttentionUNet\n",
    "from src.utils.metrics import dice_score"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n",
    "\n",
    "# Load test dataset\n",
    "test_dataset = BrainMRIDataset('../data/processed', split='test', transform=transforms.ToTensor())\n",
    "test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)\n",
    "\n",
    "# Load models\n",
    "nested_unet = NestedUNet(num_classes=1, input_channels=1).to(device)\n",
    "nested_unet.load_state_dict(torch.load('../best_nested_unet.pth', map_location=device))\n",
    "nested_unet.eval()\n",
    "\n",
    "attention_unet = AttentionUNet(num_classes=1, input_channels=1).to(device)\n",
    "attention_unet.load_state_dict(torch.load('../best_attention_unet.pth', map_location=device))\n",
    "attention_unet.eval()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def evaluate_model(model, data_loader):\n",
    "    dice_scores = []\n",
    "    with torch.no_grad():\n",
    "        for images, masks in tqdm(data_loader):\n",
    "            images, masks = images.to(device), masks.to(device)\n",
    "            outputs = model(images)\n",
    "            dice = dice_score(outputs, masks)\n",
    "            dice_scores.append(dice.item())\n",
    "    return np.mean(dice_scores)\n",
    "\n",
    "nested_unet_dice = evaluate_model(nested_unet, test_loader)\n",
    "attention_unet_dice = evaluate_model(attention_unet, test_loader)\n",
    "\n",
    "print(f\"Nested U-Net Dice Score: {nested_unet_dice:.4f}\")\n",
    "print(f\"Attention U-Net Dice Score: {attention_unet_dice:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_segmentation_results(model, data_loader, num_samples=5):\n",
    "    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))\n",
    "    \n",
    "    with torch.no_grad():\n",
    "        for i, (images, masks) in enumerate(data_loader):\n",
    "            if i >= num_samples:\n",
    "                break\n",
    "            \n",
    "            images, masks = images.to(device), masks.to(device)\n",
    "            outputs = model(images)\n",
    "            \n",
    "            image = images[0, 0].cpu().numpy()\n",
    "            mask = masks[0, 0].cpu().numpy()\n",
    "            pred = torch.sigmoid(outputs[0, 0]).cpu().numpy() > 0.5\n",
    "            \n",
    "            axes[i, 0].imshow(image, cmap='gray')\n",
    "            axes[i, 0].set_title('MRI Image')\n",
    "            axes[i, 0].axis('off')\n",
    "            \n",
    "            axes[i, 1].imshow(mask, cmap='gray')\n",
    "            axes[i, 1].set_title('Ground Truth')\n",
    "            axes[i, 1].axis('off')\n",
    "            \n",
    "            axes[i, 2].imshow(pred, cmap='gray')\n",
    "            axes[i, 2].set_title('Prediction')\n",
    "            axes[i, 2].axis('off')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "print(\"Nested U-Net Results:\")\n",
    "plot_segmentation_results(nested_unet, test_loader)\n",
    "\n",
    "print(\"Attention U-Net Results:\")\n",
    "plot_segmentation_results(attention_unet, test_loader)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}